In [1]:
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from   torchtext.vocab import GloVe
from   torchtext.legacy.data import Field
from   torch.optim import lr_scheduler

##### 构造数据集

In [2]:
# step 1. 创建字段(字段就是数据集中都有哪些特征)
# 加载pytorch中torchtext内置的IMDB电影评论数据集
#                                  1.评论内容转为小写  2.评论截取200字符 3.设置第一维度是batch编号
TEXT = torchtext.legacy.data.Field(lower=True,fix_length=200,batch_first=True)
#                                  1.不进行对标签的乱序保证一一对应
LABEL= torchtext.legacy.data.Field(sequential=False)

In [3]:
# step 2. 加载数据
# 划分数据集为训练数据集以及测试数据集并设置自定义字段
train,test = torchtext.legacy.datasets.IMDB.splits(TEXT,LABEL)

In [4]:
# 查看train数据集中的字段
# train.fields  

In [5]:
# step 3. 创建词表
# 根据train数据集创建词表，并且仅仅关注出现频率为前10000的单词 
# 其他标注为Unknown ,设置出现频率小于3的单词被抛弃
# 创建词表之后会将一个单词映射到一个固定且唯一的数值
TEXT.build_vocab(train,max_size=10000,min_freq=3) 
LABEL.build_vocab(train)

In [6]:
# TEXT.vocab.freqs        # 查看每一个单词出现的频数

In [7]:
# TEXT.vocab.stoi         # 查看词表内容  

In [8]:
# len(TEXT.vocab.stoi)    # 整个此表的长度为10002 (词表+未知值+填充值)        

In [9]:
# 利用torchtext创建文本DataLoader
train_iter,test_iter = torchtext.legacy.data.BucketIterator.splits((train,test),batch_size=16)

In [10]:
# 查看数据集中的数据
# b = next(iter(train_iter))# 每一次返回的值是文本-标签对
# print(b.text)             # 查看一批次自然语言文本(测试数据集中的一个batch)对应的数据向量
# print("\n\n")
# print(b.label)            # 查看一批次自然语言文本(测试数据集中的一个batch)对应的标签
# print("\n\n")
# print(b.text.shape)       # 查看一批次自然语言文本(测试数据集中的一个batch)的形状
# print("\n\n")
# print(b.label.shape)      # 查看一批次自然语言文本对应标签(测试数据集中的一个batch)的形状

##### 创建模型

In [11]:
# 未考虑文本顺序 -- 效果差
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # 将单词做映射 参数为: 输入词表长度 以及 用多长的向量表示一个的单词
        self.em  = nn.Embedding(len(TEXT.vocab.stoi),100) # batch*200单词 <--> batch*200*100向量
        self.fc1 = nn.Linear(200*100,1024)                # 
        self.fc2 = nn.Linear(1024,3)                      # 定义为3分类模型
    def forward(self,x):
        x = self.em(x)                                    # 文本的词向量表示
        x = x.view(x.size(0),-1)                          # 将数据扁平化
        x = self.fc1(x)
        x = F.relu(x)                                     # 激活
        x = self.fc2(x)                                   # 输出值生成
        return x

In [12]:
model = Net()
model = model.to("cuda")

In [13]:
loss_fn            = nn.CrossEntropyLoss()
optimizer          = torch.optim.Adam(model.parameters())
exp_lr_scheduler   = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
epochs             = 10

##### 训练模型

In [14]:
def fit(epoch, model, trainloader, testloader):
    correct      = 0
    total        = 0
    running_loss = 0
    
    model.train()
    for b in trainloader:
        x, y     = b.text, b.label
        if torch.cuda.is_available():
            x, y = b.text.to('cuda'), b.label.to('cuda')
        y_pred   = model(x)
        loss     = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred       = torch.argmax(y_pred, dim=1)
            correct      += (y_pred == y).sum().item()
            total        += y.size(0)
            running_loss += loss.item()
    exp_lr_scheduler.step()
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc  = correct / total
        
        
    test_correct      = 0
    test_total        = 0
    test_running_loss = 0 
    
    model.eval()
    with torch.no_grad():
        for b in testloader:
            x, y     = b.text, b.label
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred             = model(x)
            loss               = loss_fn(y_pred, y)
            y_pred             = torch.argmax(y_pred, dim=1)
            test_correct      += (y_pred == y).sum().item()
            test_total        += y.size(0)
            test_running_loss += loss.item()
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc  = test_correct / test_total  
    print('epoch: ', epoch, '\t loss： ', round(epoch_loss, 3),'\t accuracy:', round(epoch_acc, 3),'\t test_loss： ', round(epoch_test_loss, 3),'\t test_accuracy:', round(epoch_test_acc, 3))
    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc

In [15]:
train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch, model,train_iter,test_iter)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

epoch:  0 	 loss：  0.045 	 accuracy: 0.57 	 test_loss：  0.042 	 test_accuracy: 0.609
epoch:  1 	 loss：  0.023 	 accuracy: 0.832 	 test_loss：  0.04 	 test_accuracy: 0.705
epoch:  2 	 loss：  0.008 	 accuracy: 0.954 	 test_loss：  0.057 	 test_accuracy: 0.722
epoch:  3 	 loss：  0.003 	 accuracy: 0.982 	 test_loss：  0.076 	 test_accuracy: 0.718
epoch:  4 	 loss：  0.003 	 accuracy: 0.986 	 test_loss：  0.09 	 test_accuracy: 0.726
epoch:  5 	 loss：  0.002 	 accuracy: 0.991 	 test_loss：  0.088 	 test_accuracy: 0.737
epoch:  6 	 loss：  0.002 	 accuracy: 0.991 	 test_loss：  0.099 	 test_accuracy: 0.734
epoch:  7 	 loss：  0.001 	 accuracy: 0.997 	 test_loss：  0.099 	 test_accuracy: 0.738
epoch:  8 	 loss：  0.0 	 accuracy: 1.0 	 test_loss：  0.102 	 test_accuracy: 0.74
epoch:  9 	 loss：  0.0 	 accuracy: 1.0 	 test_loss：  0.108 	 test_accuracy: 0.741
